In [1]:
import numpy as np
import random

# Магический квадрат

Размер магического квадрата

In [2]:
phrase = 'Амилопектин состоит из молекул глюкозы'

In [3]:
Size = 5

## Создание Магического квадрата

In [4]:
def createMagicSquare(size):
    N = size
    magic_square = np.zeros((N,N), dtype=int)

    n = 1
    i, j = 0, N//2

    while n <= N**2:
        magic_square[i, j] = n
        n += 1
        newi, newj = (i-1) % N, (j+1)% N
        if magic_square[newi, newj]:
            i += 1
        else:
            i, j = newi, newj
    
    return magic_square

In [5]:
magicSquare = createMagicSquare(Size)

In [6]:
magicSquare

array([[17, 24,  1,  8, 15],
       [23,  5,  7, 14, 16],
       [ 4,  6, 13, 20, 22],
       [10, 12, 19, 21,  3],
       [11, 18, 25,  2,  9]])

## Фраза

Поскольку длина нашей фразы больше размерности квадрата,
разобьем фразу на несколько квадратов

In [7]:
def chunkstring(string, length):
    return (string[0+i:length+i] for i in range(0, len(string), length))

In [8]:
phrasesBlocks = list(chunkstring(phrase, Size**2))

In [9]:
phrasesBlocks

['Амилопектин состоит из мо', 'лекул глюкозы']

## Подстановка в квадрат

In [10]:
def turnInMagicSquare(square, phrase):
    sz = len(square)
    phraseSquare = np.empty((sz, sz), dtype='str')
    for row in range(sz):
        for col in range(sz):
            phraseSquare[row][col] = phrase[square[row][col] - 1]
    return phraseSquare

In [11]:
cryptSquares = []
for block in phrasesBlocks:
    while len(block) < Size ** 2:
        block += ' '
    cryptSquares.append(turnInMagicSquare(magicSquare, block))

In [12]:
cryptSquares

[array([['о', 'м', 'А', 'к', 'с'],
        [' ', 'о', 'е', 'о', 'т'],
        ['л', 'п', 'с', ' ', 'з'],
        ['и', ' ', 'т', 'и', 'и'],
        ['н', 'и', 'о', 'м', 'т']], dtype='<U1'),
 array([[' ', ' ', 'л', 'л', ' '],
        [' ', 'л', 'г', ' ', ' '],
        ['у', ' ', 'ы', ' ', ' '],
        ['к', 'з', ' ', ' ', 'к'],
        ['о', ' ', ' ', 'е', 'ю']], dtype='<U1')]

## Раскодировка

In [13]:
decryptedBlocks = ''
for crypted in cryptSquares:
    decrBlock = ''
    for decNum in range(Size**2):
        num = decNum + 1
        for i in range(Size):
            for j in range(Size):
                if magicSquare[i][j] == num:
                    decrBlock += crypted[i][j]
    decryptedBlocks += decrBlock

In [14]:
print(decryptedBlocks)

Амилопектин состоит из молекул глюкозы            


# RSA

In [15]:
p = 167
q = 661
word = 'Амилопектин'

## Расширенный алгоритм Евклида

In [16]:
def bezout(a, b):
    '''An implementation of extended Euclidean algorithm.
    Returns integer x, y and gcd(a, b) for Bezout equation:
        ax + by = gcd(a, b).
    '''
    x, xx, y, yy = 1, 0, 0, 1
    while b:
        q = a // b
        a, b = b, a % b
        x, xx = xx, x - xx*q
        y, yy = yy, y - yy*q
    return (x, y, a)

In [17]:
def calcEuler(p, q):
    return (p - 1) * (q - 1)

In [18]:
n = p * q

In [19]:
def chooseE(n):
    euler = calcEuler(p , q)
    variants = [i for i in range(2, n - 1)]
    length = len(variants)
    while(length):
        index = random.randint(0, length)
        nod = bezout(euler, variants[index])[2]
        if nod == 1:
            return variants[index]
        else:
            del(variants[index])
            length -= 1

In [20]:
openKey = chooseE(n)

In [21]:
openKey

32777

## Проверка e

In [22]:
bezout(openKey, calcEuler(p, q))[2]

1

In [23]:
def chooseD(e):
    euler = calcEuler(p , q)
    bzt = bezout(euler, e)
    if bzt[1] >= 0:
        return bzt[1]
    else:
        return euler + bzt[1]

In [24]:
closeKey = chooseD(openKey)

In [25]:
closeKey

70913

## Проверка d 

In [26]:
(openKey * closeKey) % calcEuler(p , q)

1

## Шифрование

In [27]:
codes = [ord(i) for i in word]

In [28]:
codes

[1040, 1084, 1080, 1083, 1086, 1087, 1077, 1082, 1090, 1080, 1085]

In [29]:
crypted = [pow(i, openKey, n) for i in codes]

In [30]:
crypted

[102392, 23152, 65055, 97271, 40837, 106941, 57633, 42583, 8814, 65055, 55188]

# Дешифрование

In [31]:
decrypted = [pow(i, closeKey, n) for i in crypted]

In [32]:
decrypted

[1040, 1084, 1080, 1083, 1086, 1087, 1077, 1082, 1090, 1080, 1085]

In [33]:
result = ''
for i in decrypted:
    result += chr(i)

In [34]:
result

'Амилопектин'